# European Data Job Market Analysis (2023–Q2 2025)

This notebook documents the data preparation pipeline used to build the Looker Studio dashboard.

**Main steps**
1. Load source dataset (CSV / BigQuery export)
2. Exploratory checks (missing values, basic distributions)
3. Geo-enrichment (city → country code, lat/lon)
4. Cleaning & feature engineering (role classification, schedule normalization)
5. Outputs exported for BI (clean dataset + skills tables)

> Note: BigQuery / Colab authentication code was removed for GitHub portability.


# Data Analyst Job project

heck dataset from hugging face - luke Barousse : data_jobs.csv (2023 till June 2025)


In [ ]:
import pandas as pd
import datetime as dt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np

# # Quick Overview

In [ ]:
df.info()

## # How tables are joined and which countries are in the data set

In [ ]:
'''SQL QUery : WITH skills_agg AS (
  SELECT
    skills.job_id,
    STRING_AGG(DISTINCT skd.skills, ', ') AS skills_list,
    STRING_AGG(DISTINCT skd.type, ', ') AS skills_types
  FROM `jobprojectlewagon.full_luke_dataset_raw.skills_job_dim_raw` AS skills
  JOIN `jobprojectlewagon.full_luke_dataset_raw.skills_dim_raw` AS skd
    USING (skill_id)
  GROUP BY skills.job_id
)

SELECT
  job.job_id,
  job.job_title_short,
  job.job_title,
  job.job_location,
  job.job_country,
  job.job_posted_date,
  job.job_schedule_type,
  job.job_work_from_home,
  job.job_no_degree_mention,
  job.job_via,
  job.salary_hour_avg,
  job.salary_rate,
  job.salary_year_avg,
  comp.company_id,
  comp.name AS company_name,
  skills_agg.skills_list,
  skills_agg.skills_types
FROM `jobprojectlewagon.full_luke_dataset_raw.job_postings_fact_raw` AS job
LEFT JOIN `jobprojectlewagon.full_luke_dataset_raw.company_dim_raw` AS comp
  USING (company_id)
LEFT JOIN skills_agg
  ON job.job_id = skills_agg.job_id
WHERE job.job_country IN (
  'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
  'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
  'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia',
  'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland',
  'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
  'United Kingdom', 'Switzerland', 'Norway', 'Iceland', 'Liechtenstein',
  'Albania', 'Bosnia and Herzegovina', 'Serbia', 'Montenegro', 'North Macedonia',
  'Moldova', 'Ukraine', 'Belarus'
);'''

## Exploratory Analysis

## # Null & counts

In [ ]:
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

In [ ]:
months_nb = df['job_posted_date'].dt.month

#offers count by month
offers_by_month = months_nb.value_counts().sort_index()

print(f"nb of offers by month : {offers_by_month}")


In [ ]:
years_nb = df['job_posted_date'].dt.year

#offers count by year
offers_by_year = years_nb.value_counts().sort_index()

print(f"nb of offers by year : {offers_by_year}")

In [ ]:
day_names = df['job_posted_date'].dt.day_name()

# Compte les offres pour chaque jour de la semaine
offers_by_weekday = day_names.value_counts()

print("Nombre d'offres par jour de la semaine :")
print(offers_by_weekday)

## # Dataviz

In [ ]:
df.hist(figsize=(12, 10), bins=30)
plt.suptitle("distributions")
plt.show()

In [ ]:
#top 10 job titles for the categories that Luke decided on

plt.figure(figsize=(10,5))
sns.countplot(y=df['job_title_short'], order=df['job_title_short'].value_counts().index[:10])
plt.title("top 10 job titles")
plt.xlabel("count")
plt.ylabel("job title short")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y=df['job_country'], order=df['job_country'].value_counts().index[:10])
plt.title("top 10 countries")
plt.xlabel("count")
plt.ylabel("job countries")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y=df['job_location'], order=df['job_location'].value_counts().index[:10])
plt.title("top 10 locations")
plt.xlabel("count")
plt.ylabel("job locations")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y=df['job_via'], order=df['job_via'].value_counts().index[:10])
plt.title("top 10 sites")
plt.xlabel("count")
plt.ylabel("job sites")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y=df['company_name'], order=df['company_name'].value_counts().index[:10])
plt.title("top 10 companies")
plt.xlabel("count")
plt.ylabel("job companies")
plt.show()

In [ ]:
px.histogram(
    df,
    x="job_country",
    color="job_work_from_home",
    barmode="group",
    title="work from home by country",
    labels={
        "job_country": "country",
        "job_work_from_home": "work from home or not?"
    },
    color_discrete_sequence=px.colors.qualitative.Vivid
)

In [ ]:
px.histogram(
    df,
    x="job_location",
    color="job_work_from_home",
    barmode="group",
    title="work from home by location",
    labels={
        "job_location": "location",
        "job_work_from_home": "work from home or not?"
    },
    color_discrete_sequence=px.colors.qualitative.Vivid
)

## # Geoloc table

### # Split job location to create a new column city.
* If nulls -> 'Unknown'
* If only country name -> 'Unknown'
* If no city but only '(other)' -> 'Unknown'
* Erase spaces

In [ ]:
def extract_city(job_location):
  # Null and NaN
  if pd.isna(job_location):
    return 'Unknown'

  # normalize spaces
  loc = job_location.strip()

  # no comma, no city
  if ',' not in loc:
    return 'Unknown'

  #extract first part -> city
  city = loc.split(',')[0].strip()

  #Parentheses -> (others) case
  if city.startswith('('):
    return 'Unknown'

  #remove duplicated spaces:
  city = re.sub(r"\s+", " ", city)

  return city

df['job_city'] = df['job_location'].apply(extract_city)



### # Iso column country code step

In [ ]:
country_to_iso = {
    'Austria': 'AT',
    'Belgium': 'BE',
    'Bulgaria': 'BG',
    'Croatia': 'HR',
    'Cyprus': 'CY',
    'Czech Republic': 'CZ',
    'Denmark': 'DK',
    'Estonia': 'EE',
    'Finland': 'FI',
    'France': 'FR',
    'Germany': 'DE',
    'Greece': 'GR',
    'Hungary': 'HU',
    'Ireland': 'IE',
    'Italy': 'IT',
    'Latvia': 'LV',
    'Lithuania': 'LT',
    'Luxembourg': 'LU',
    'Malta': 'MT',
    'Netherlands': 'NL',
    'Poland': 'PL',
    'Portugal': 'PT',
    'Romania': 'RO',
    'Slovakia': 'SK',
    'Slovenia': 'SI',
    'Spain': 'ES',
    'Sweden': 'SE',
    'United Kingdom': 'GB',
    'Switzerland': 'CH',
    'Norway': 'NO',
    'Iceland': 'IS',
    'Liechtenstein': 'LI',
    'Albania': 'AL',
    'Bosnia and Herzegovina': 'BA',
    'Serbia': 'RS',
    'Montenegro': 'ME',
    'North Macedonia': 'MK',
    'Moldova': 'MD',
    'Ukraine': 'UA',
    'Belarus': 'BY'
}

In [ ]:
df.drop(['city'], axis=1, inplace=True)

In [ ]:
df['country_code'] = df['job_country'].map(country_to_iso).fillna('Unknown')

### # Add geonames data (cities > 1000 hab)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:


columns = [
    "geonameid", "name", "asciiname", "alternatenames", "lat", "lon",
    "feature_class", "feature_code", "country_code", "cc2",
    "admin1", "admin2", "admin3", "admin4",
    "population", "elevation", "dem", "timezone", "modification_date"
]

geo = pd.read_csv(
    '/content/drive/MyDrive/Data/Wagon/DataJob/cities1000.txt',
    sep='\t',
    header=None,
    names=columns,
    engine='python')

In [ ]:
print(geo.columns)
print(geo.head(3))

In [ ]:
geo.head(5)

### # merge city and lat, lon from Geo Names

In [ ]:
# Normalization
df['name'] = df['job_city'].str.title().str.strip()
df['country_code'] = df['country_code'].str.upper().str.strip()
geo['name'] = geo['name'].str.title().str.strip()
geo['country_code'] = geo['country_code'].str.upper().str.strip()

# Merge
df_geoloc = df.merge(
    geo[['name', 'country_code', 'lat', 'lon']],
    on=['name', 'country_code'],
    how='left'
)

# Missing data ?
print(df_geoloc[df_geoloc['lat'].isna()][['name', 'country_code']].value_counts().drop_duplicates())

In [ ]:
df_geoloc.drop(['salary_hour_avg','salary_rate','salary_year_avg'], axis=1, inplace=True)
df_geoloc.head(1)

In [ ]:
df_geoloc.head(5)

In [ ]:
df_geoloc_path = "/content/drive/MyDrive/Data/Wagon/DataJob/geoloc.csv"
df_geoloc.to_csv(df_geoloc_path, index=False)

print("Fichiers exportés :")
print(df_geoloc_path)


# # Cleaning

## # Job Title

In [ ]:
df_job = df[['job_id', 'job_title', 'skills_list']]
df_job.info()

### # First classification

### # Counter-proposal - Kamil reorder

### # Proposition augmented with AI help

### # Test with no skills check, only job title

### # Test without skills and priority

### # Final classification

## Role classification (rule-based)

We classify each job posting into a role category using **explainable regex rules** applied to:
1) `job_title` (primary signal)  
2) `skills_list` (secondary signal for unresolved cases)

The rule set is stored in `src/role_rules.py` to keep this notebook readable and reusable.

### # Checking results

In [ ]:
from src.role_rules import categorize_dataframe

df_job["job_category"] = categorize_dataframe(df_job, title_col="job_title", skills_col="skills_list")

Test on a new dataframe

## # Drop columns

In [ ]:
## Drop columns

cols_to_drop = [
    "job_title_short",
    "salary_hour_avg",
    "salary_rate",
    "salary_year_avg",
    "company_id"
]

df_clean = df_clean.drop(columns=cols_to_drop, errors="ignore")
df_clean.head(3)

In [ ]:
df_clean['job_category'].value_counts()

In [ ]:
# --- Quick quality report (sanity check) ---
role_counts = df_clean["job_category"].value_counts(dropna=False)
role_share = (df_clean["job_category"].value_counts(normalize=True, dropna=False) * 100).round(2)

report = pd.DataFrame({"count": role_counts, "share_%": role_share})

unknown_labels = {"undefined", "unknown", "uncategorized", None, pd.NA, float("nan")}
# Compute unknown share robustly
unknown_mask = df_clean["job_category"].astype("string").str.lower().isin({"undefined", "unknown", "uncategorized"})
unknown_pct = round(100 * unknown_mask.mean(), 2)

print(f"Rows: {len(df_clean):,}")
print(f"Unknown/undefined roles: {unknown_pct}%")
display(report)


In [ ]:
# Remove nulls

cols_unknown = ["job_no_degree_mention", "job_via", "job_location", "company_name", "skills_list", "skills_types"]
for col in cols_unknown:
    df_clean[col] = df_clean[col].fillna("unknown")

# remove "via"
df_clean["job_via"] = df_clean["job_via"].astype(str).str.replace(r"\bvia\b", "", regex=True).str.strip()

In [ ]:
df_clean.isnull().sum()

In [ ]:
df_clean["job_posted_date"] = pd.to_datetime(df_clean["job_posted_date"], errors="coerce")

df_clean["job_posted_year"] = df_clean["job_posted_date"].dt.year
df_clean["job_posted_month"] = df_clean["job_posted_date"].dt.month

## # Order of columns

In [ ]:
# organize column order


def move_after(col_list, col_to_move, after_col):
    if col_to_move in col_list:
        col_list.remove(col_to_move)
        idx = col_list.index(after_col) + 1
        col_list.insert(idx, col_to_move)
    return col_list

cols = list(df_clean.columns)

# 1) job_category after job_title
cols = move_after(cols, "job_category", "job_title")

# 2)  job_posted_month after job_posted_date
cols = move_after(cols, "job_posted_month", "job_posted_date")

# 3) job_posted_year after job_posted_month
cols = move_after(cols, "job_posted_year", "job_posted_month")

# Appliquer réorganisation
df_clean = df_clean[cols]

In [ ]:
df_clean.head()

## # New dataframe - Skills

In [ ]:
df_clean['skills_list'] = df_clean['skills_list'].fillna('').astype(str)

df_skills = (
    df_clean[['job_id', 'skills_list']]
        .assign(skill=lambda x: x.skills_list.str.split(','))
        .explode('skill')
        .assign(skill=lambda x: x['skill'].str.strip())  # nettoyage
        .query("skill != ''")  # on enlève les vides
        .drop(columns=['skills_list'])
        .reset_index(drop=True)
)

df_skills['skill'].value_counts()

In [ ]:
df_skills.head()

In [ ]:
df_skills.to_csv("df_skills.csv", index=False, encoding="utf-8")

## # New df Skills_types

In [ ]:
df_skill_types = (
    df_clean[['job_id', 'skills_types']]
        .assign(skill_type=lambda x: x.skills_types.fillna('').astype(str).str.split(','))
        .explode('skill_type')
        .assign(skill_type=lambda x: x['skill_type'].str.strip().str.lower())
        .query("skill_type != ''")
        .drop(columns=['skills_types'])
        .reset_index(drop=True)
)

df_skill_types['skill_type'].value_counts()

In [ ]:
df_skill_types.to_csv("df_skill_types.csv", index=False, encoding="utf-8")

## # Categorize better job_schedule_type

In [ ]:
# Mapping des types de jobs
map_job_type_whole_column = {
    "full-time": "full-time",
    "full time": "full-time",
    "pekerjaan tetap": "full-time",
    "part-time": "part-time",
    "part time": "part-time",
    "contract": "contract",
    "contractor": "contract",
    "temporary": "temporary",
    "temp": "temporary",
    "temp work": "temporary",
    "per diem": "temporary",
    "internship": "internship",
    "intern": "internship",
    "freelance": "freelance",
    "volunteer": "other",
}

# Fonction de catégorisation
def categorise_job_type(text):
    if pd.isna(text):
        return ["other"]

    text = text.lower()
    found = []

    for pattern, category in map_job_type_whole_column.items():
        if re.search(r"\b" + re.escape(pattern) + r"\b", text):
            found.append(category)

    return found if found else ["other"]

# Application
df_clean["job_scheduled"] = df_clean["job_schedule_type"].apply(categorise_job_type)
df_clean

# Comptage des catégories (explosion des listes)
counts = df_clean["job_scheduled"].explode().value_counts()
print(counts)

In [ ]:
cols_to_drop = [
    "job_schedule_type",
    "job_categories",
]

df_clean = df_clean.drop(columns=cols_to_drop, errors="ignore")
df_clean.head(3)


In [ ]:
df_clean.to_csv("df_data_job_clean.csv", index=False, encoding="utf-8")